In [ ]:
from scipy import misc, ndimage, signal
from sklearn.model_selection  import train_test_split
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from time import time
import time as tm
import datetime
from operator import itemgetter
import glob
from skimage.util.shape import view_as_blocks
from keras.utils import np_utils
from keras.utils import to_categorical

In [ ]:
!nvidia-smi
#!kill PIDnumber

In [ ]:
path_folder = "./"
path_img_base = './images'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

In [ ]:
################################################## 30 original SRM FILTERS
srm_weights = np.load(path_folder+'/SRM_Kernels1.npy') 
print (srm_weights.shape)
################################################## bias
biasSRM=numpy.ones(30)
################################################## TLU ACTIVATION FUNCTION
def Tanh3(x):
    tanh3 = tf.keras.activations.tanh(x)*3
    return tanh3
##################################################

In [ ]:
path_model = path_folder+"../Trained_models_SRMbifurcation/GBRAS-Net_WOW04_SRMbifurcation.hdf5"
model_GBRAS = tf.keras.models.load_model(path_model, custom_objects={'Tanh3':Tanh3}, compile=True)

path_model = path_folder+"../Trained_models_SRMbifurcation/VGG16stego_WOW04_SRMbifurcation.hdf5"
model_VGG16 = tf.keras.models.load_model(path_model, custom_objects={'Tanh3':Tanh3}, compile=True)

path_model = path_folder+"../Trained_models_SRMbifurcation/VGG19stego_WOW04_SRMbifurcation.hdf5"
model_VGG19 = tf.keras.models.load_model(path_model, custom_objects={'Tanh3':Tanh3}, compile=True)

path_model = path_folder+"../Trained_models_SRMbifurcation/YEDROUDJ-Net_WOW04_SRMbifurcation.hdf5"
model_YEDROUDJ = tf.keras.models.load_model(path_model, custom_objects={'Tanh3':Tanh3}, compile=True)

path_model = path_folder+"../Trained_models_SRMbifurcation/YE-Net_WOW04_SRMbifurcation.hdf5"
model_YE = tf.keras.models.load_model(path_model, custom_objects={'Tanh3':Tanh3}, compile=True)

path_model = path_folder+"../Trained_models_SRMbifurcation/XU-Net_WOW04_SRMbifurcation.hdf5"
model_XU = tf.keras.models.load_model(path_model, custom_objects={'Tanh3':Tanh3}, compile=True)


In [ ]:
path_stego = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/WOW/0.4bpp/stego"
path_cover = "/media/ia/Datos1/DocReinel/Steganalysis/H_Brayan_A_Arteaga/Frankenstein_Project/GBRAS-Net/DATABASES/BOSSbase-1.01/cover"

n=256
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in sorted(files):
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X

Xc_ = load_images(path_cover+'/*.pgm') ##COVER IMAGES
Xs_ = load_images(path_stego+'/*.pgm') ##STEGO IMAGES
X_  = (numpy.vstack((Xc_, Xs_)))
Xt_ = (numpy.hstack(([0]*len(Xc_), [1]*len(Xs_))))
Xt_ = np_utils.to_categorical(Xt_, 2)
X_  = np.rollaxis(X_,1,4)  #channel axis shifted to last axis

print("Total image data and labels",X_.shape,Xt_.shape)
#Cover hasta las 10000 ##Train hasta las 4000 ##Valid hasta de las 4000 a las 5000 ##Test de las 5000 a las 10000
X_train = np.concatenate([X_[0:4000],X_[10000:14000]],axis=0)
X_valid = np.concatenate([X_[4000:5000],X_[14000:15000]],axis=0)
X_test  = np.concatenate([X_[5000:10000],X_[15000:20000]],axis=0)
y_train = np.concatenate([Xt_[0:4000],Xt_[10000:14000]],axis=0)
y_valid = np.concatenate([Xt_[4000:5000],Xt_[14000:15000]],axis=0)
y_test  = np.concatenate([Xt_[5000:10000],Xt_[15000:20000]],axis=0)
#Controled randomized data for training
X_dat0, X_dat1, y_dat0, y_dat1 = train_test_split(X_train, y_train, test_size=0.50, random_state=64) 
X_train = np.concatenate([X_dat0,X_dat1],axis=0) 
y_train = np.concatenate([y_dat0,y_dat1],axis=0) 
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
loss,accuracy = model_GBRAS.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_VGG16.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_VGG19.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_YEDROUDJ.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_YE.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')

loss,accuracy = model_XU.evaluate(X_test,y_test,verbose=0) #Test BOSSbase 1.01 
print(f'Loss={loss:.4f} and Accuracy={accuracy:0.3f}')


In [ ]:
def display_filters(w, figsize=(15,15)):
    plt.figure(figsize=figsize)
    w = (w-np.min(w))/(np.max(w)-np.min(w))
    cont=0
    for i in range(w.shape[-1]):
        for j in range(w.shape[-2]):
            cont=cont+1
            plt.subplot(w.shape[-2],w.shape[-1],cont)
            plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
            plt.axis("off")

In [ ]:
models = [model_GBRAS,model_VGG16,model_VGG19,model_YEDROUDJ,model_YE,model_XU]

In [ ]:
models_name = ["model_GBRAS","model_VGG16","model_VGG19","model_YEDROUDJ","model_YE","model_XU"]

In [ ]:
path_lf = './images/learned_filters'
if not os.path.exists(path_lf):
    os.makedirs(path_lf)
    
iter = 0

for m in models:
    model = m
    w1 = model.get_weights()[2]
    print (w1.shape)
    type(display_filters(w1))
    
    print(w1[:,:,0,0])
    
    name = models_name[iter]
    np.save(path_lf+'/SRM_learned_filters_'+name,w1) 
    
    iter = iter+1

In [ ]:
path_img_base = './images/learned_filters_graph'
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)

In [ ]:
iter = 0
for path in glob.glob(path_lf+"/*.npy"):
    srm_weights = np.load(path)
    name = models_name[iter]
    def display_filters(w, figsize=(30,30)):
        plt.figure(figsize=figsize)
        w = (w-np.min(w))/(np.max(w)-np.min(w))
        cont=0
        for i in range(w.shape[-1]):
            for j in range(w.shape[-2]):
                cont=cont+1
                plt.subplot(5,6,cont)
                plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
                plt.axis("off")
                plt.savefig(path_img_base+'/SRM_'+name+'.pdf', format='pdf') 
                plt.savefig(path_img_base+'/SRM_'+name+'.svg', format='svg') 
    
    iter = iter+1
    display_filters(srm_weights)
    print(srm_weights.shape)  

In [ ]:
srm_weights = np.load('SRM_Kernels1.npy')
name = "original"
def display_filters(w, figsize=(30,30)):
    plt.figure(figsize=figsize)
    w = (w-np.min(w))/(np.max(w)-np.min(w))
    cont=0
    for i in range(w.shape[-1]):
        for j in range(w.shape[-2]):
            cont=cont+1
            plt.subplot(5,6,cont)
            plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
            plt.axis("off")
            plt.savefig(path_img_base+'/SRM_'+name+'.pdf', format='pdf') 
            plt.savefig(path_img_base+'/SRM_'+name+'.svg', format='svg') 
                
display_filters(srm_weights)
print(srm_weights.shape)  